# 使用BERT进行中文情感分类




# 一、数据获取

## 访问Google Drive上数据

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd

In [0]:
path="/content/drive/My Drive/Colab Notebooks/NLP/04_BERT/"
train=pd.read_csv(path+"nCoV_100k_train.labled.csv")
test=pd.read_csv(path+"nCov_10k_test.csv")

In [0]:
train.head()

,微博id,微博发布时间,发布人账号,微博中文内容,微博图片,微博视频,情感倾向
0,4456072029125500,01月01日 23:50,存曦1988,写在年末冬初孩子流感的第五天，我们仍然没有忘记热情拥抱这2020年的第一天。带着一丝迷信，早...,['https://ww2.sinaimg.cn/orj360/005VnA1zly1gah...,[],0
1,4456074167480980,01月01日 23:58,LunaKrys,开年大模型…累到以为自己发烧了腰疼膝盖疼腿疼胳膊疼脖子疼#Luna的Krystallife#?,[],[],-1
2,4456054253264520,01月01日 22:39,小王爷学辩论o_O,�偳癯空饩褪俏业�，爹，发烧快好，毕竟美好的假期拿来养病不太好，假期还是要好好享受快乐，爹，...,['https://ww2.sinaimg.cn/thumb150/006ymYXKgy1g...,[],1
3,4456061509126470,01月01日 23:08,芩鎟,新年的第一天感冒又发烧的也太衰了但是我要想着明天一定会好的?,['https://ww2.sinaimg.cn/orj360/005FL9LZgy1gah...,[],1
4,4455979322528190,01月01日 17:42,changlwj,问：我们意念里有坏的想法了，天神就会给记下来，那如果有好的想法也会被记下来吗？答：那当然了。...,[],[],1


In [0]:
columns=['id','date','user','content','image','vedio','target']
train.columns=columns
test.columns=columns[:-1]
train.drop(['id','date','user','image','vedio'],axis = 1,inplace = True)
train.head()

,content,target
0,写在年末冬初孩子流感的第五天，我们仍然没有忘记热情拥抱这2020年的第一天。带着一丝迷信，早...,0
1,开年大模型…累到以为自己发烧了腰疼膝盖疼腿疼胳膊疼脖子疼#Luna的Krystallife#?,-1
2,�偳癯空饩褪俏业�，爹，发烧快好，毕竟美好的假期拿来养病不太好，假期还是要好好享受快乐，爹，...,1
3,新年的第一天感冒又发烧的也太衰了但是我要想着明天一定会好的?,1
4,问：我们意念里有坏的想法了，天神就会给记下来，那如果有好的想法也会被记下来吗？答：那当然了。...,1


## 预处理数据

### 去除空值

In [0]:
train[train.isnull().T.any().T]  #找出含有nan的所有行

,content,target
850,NaN,1
1226,NaN,0
1362,NaN,1
2042,NaN,0
2982,NaN,1
...,...,...
97739,NaN,0
98243,NaN,0
98998,NaN,0
99055,NaN,0


In [0]:
train.dropna(axis=0, how='any', inplace=True)

In [0]:
train[train.isnull().T.any().T]  #找出含有nan的所有行

,content,target


In [0]:
len(test[test.isnull().T.any().T])
#测试集37个为Nan值的数据

37

### 去除特殊符号

In [0]:
train['content'][15:25]

15    现在是北京时间11点38分我发了一个晚上的烧，难受死了，去给自己弄个冷毛巾吧，哇第一次发烧发...
16                        2020第一天就发烧，今年怕是可以过呢红红火火了。2芒市?
17                                       新年第一天在发烧当中过去了?
18                                        2020第一天就发烧很丧?
19    ??//@朴素善xi:是的，冷暖自知//@·飛行島嶼·:他们年少相识一起成长一起出道互相照顾...
20    可怜的大宝，今天咳嗽发烧，还被我训，被我打，为什么要乱扔东西呢？为什么要打人呢？妈妈骂你打你...
21    //@木堇LMWH:【??李汶翰??】???青春有你第一C位出道???全能ACE舞台王者??...
22                           打喷嚏咳嗽不捂住猪嘴的祝你发烧感冒到2030cnm?
23    晚上回家阿姨告知苗小苗发烧了，去了医院验血，嗓子和肺都没事，血象有点高，开了头孢。我们等候之...
24                   我太难了又喉咙痛，小发烧还要准备三个考试，还都是闭卷考，还都那么难?
Name: content, dtype: object

In [0]:
import re
def clean(text):
    text = re.sub(r"(回复)?(//)?\s*@\S*?\s*(:| |$)", " ", text)  # 去除正文中的@和回复/转发中的用户名
    text = re.sub(r"\[\S+\]", "", text)      # 去除表情符号
    # text = re.sub(r"#\S+#", "", text)      # 保留话题内容
    URL_REGEX = re.compile(
        r'(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))',
        re.IGNORECASE)
    text = re.sub(URL_REGEX, "", text)       # 去除网址
    text = text.replace("转发微博", "")       # 去除无意义的词语
    text = re.sub(r"\s+", " ", text) # 合并正文中过多的空格
    return text.strip()

In [0]:
train['content']=train['content'].apply(clean)

In [0]:
train['content'][15:25]

15    现在是北京时间11点38分我发了一个晚上的烧，难受死了，去给自己弄个冷毛巾吧，哇第一次发烧发...
16                        2020第一天就发烧，今年怕是可以过呢红红火火了。2芒市?
17                                       新年第一天在发烧当中过去了?
18                                        2020第一天就发烧很丧?
19    ?? 是的，冷暖自知 他们年少相识一起成长一起出道互相照顾经历了我看不到的岁月和曲折发烧感冒...
20    可怜的大宝，今天咳嗽发烧，还被我训，被我打，为什么要乱扔东西呢？为什么要打人呢？妈妈骂你打你...
21    【??李汶翰??】???青春有你第一C位出道???全能ACE舞台王者???李汶翰全能ACE?...
22                           打喷嚏咳嗽不捂住猪嘴的祝你发烧感冒到2030cnm?
23    晚上回家阿姨告知苗小苗发烧了，去了医院验血，嗓子和肺都没事，血象有点高，开了头孢。我们等候之...
24                   我太难了又喉咙痛，小发烧还要准备三个考试，还都是闭卷考，还都那么难?
Name: content, dtype: object

In [0]:
len(train[train.isnull().T.any().T])

0

In [0]:
test.fillna("", inplace = True)#以0填充

In [0]:
test['content']=test['content'].apply(clean)

### 去除噪音标签

In [0]:
train['target'].value_counts()

0     57287
1     25374
-1    16899
9         1
-         1
10        1
4         1
-2        1
·         1
Name: target, dtype: int64

In [0]:
train_1=train.loc[(train['target']=='-1')] 
train0=train.loc[(train['target']=='0')] 
train1=train.loc[(train['target']=='1')] 
train = pd.concat([train_1,train_1,train_1,train0,train1,train1])

In [0]:
train['target'].value_counts()

0     57287
1     50748
-1    50697
Name: target, dtype: int64

In [0]:
train.reset_index(drop=True,inplace=True)
train.head()

,content,target
0,开年大模型…累到以为自己发烧了腰疼膝盖疼腿疼胳膊疼脖子疼#Luna的Krystallife#?,-1
1,发高烧反反复复，眼睛都快睁不开了。今天室友带我去看，还在发烧中建议我输液，我拒绝了。给我打针...,-1
2,明天考试今天发烧跨年给我跨坏了？？2兰州·兰州交通大学?,-1
3,我真的服了xkh昨天vv去和她说自己不舒服，描述了症状她说啊你这不是感冒没有发烧没事的晚上一...,-1
4,一个人在外就对自己好一点吧，一个人过生日应该没什么尴尬的吧，今年的惊喜来自东哥2020第一天...,-1


In [0]:
train=train.sample(frac=1)
train.head()

,content,target
90591,太惨了也，今年不会还是我的本命年吧又发烧了已经退了，已经可以重新开心的玩耍了?,0
86153,【#钟南山表示武汉还没有停止人传人#】钟南山：全国的确诊病例已经出现下降的趋势，目前武汉依然...,0
82656,转发。一定要注意打喷嚏的礼仪，喷嚏发出的气溶胶可以播散10米，“威力”巨大。随身携带纸巾，方...,0
124473,卧槽许三多去钢七连了！！军队的钢刀！军人的剔骨刀！许三多！战斗吧！！！！好燃啊！?,1
14442,闫某某和任某某两名同学擅自提前返校，并且返校后未及时向学院报告，属于瞒报个人信息行为。根据《...,-1


## 划分训练集和验证集

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split( train["content"], train["target"], test_size=0.1, random_state=42)

# 二、BERT 训练模型

In [0]:
!pip install --upgrade tensorflow
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow_hub as hub
import numpy as np

     |████████████████████████████████| 421.8MB 33kB/s 
     |████████████████████████████████| 3.9MB 34.3MB/s 
     |████████████████████████████████| 450kB 66.0MB/s 
ERROR: tensorflow-federated 0.12.0 has requirement tensorflow-addons~=0.7.0, but you'll have tensorflow-addons 0.8.3 which is incompatible.
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorflow 1.15.0
    Uninstalling tensorflow-1.15.0:
      Successfully uninstalled tensorflow-1.15.0


In [0]:
!wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py
!pip install sentencepiece
import tokenization

     |████████████████████████████████| 1.0MB 4.7MB/s 


## 2.1 用于编码和构建模型的函数

编码函数将文本进行字编码、位置编码、片段编码

In [0]:
def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    print(texts)
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

构建模型，一个简单的baseline，经过BERT层之后，利用全连接层转到一个3分类的应用。

自定义二分类的f1值函数,但keras是基于batch计算的，官方认为如果在batch基础上计算f1是具有误导性的，所以移除了f1等评价矩阵。

In [0]:
from keras import backend as K

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

Using TensorFlow backend.


在一次epoch结束之后计算相应的f1值(适合多分类)

In [0]:
import numpy as np
from keras.callbacks import Callback
from sklearn.metrics import f1_score, precision_score, recall_score
class Metrics(Callback):
    def __init__(self, val_data):
        super().__init__()
        self.validation_data = val_data
        
    def on_train_begin(self, logs={}):
        #print("on train begin", len(self.validation_data))
        self.val_f1s = []
        self.val_recalls = []
        self.val_precisions = []

    def on_epoch_end(self, epoch, logs={}):
        #print("  on epoch end :", len(self.validation_data))
        #进行预测
        xVal, yVal = self.validation_data
        val_pred = np.argmax(np.asarray(self.model.predict(xVal)), axis = -1)
        val_true = yVal

        #计算f1值
        _val_f1 = f1_score(val_true, val_pred, average='macro')
        _val_recall = recall_score(val_true, val_pred, average='macro')
        _val_precision = precision_score(val_true, val_pred, average='macro')
        self.val_f1s.append(_val_f1)
        self.val_recalls.append(_val_recall)
        self.val_precisions.append(_val_precision)
        print('— val_f1: %f — val_precision: %f — val_recall %f' %(_val_f1, _val_precision, _val_recall))
        return

搭建模型的方法

In [0]:
def build_model(bert_layer, max_len=512):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    _, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    clf_output = sequence_output[:, 0, :]
    out = Dense(3, activation='sigmoid')(clf_output)
    
    model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(Adam(lr=2e-6), loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])
    
    return model

## 2.2 加载模型

从th-hub中加载一个BERT的预训练模型，并且将它作为Leras模型中的一层

In [0]:
#load BERT as a KerasLayer
%time
#module_url = "https://tfhub.dev/tensorflow/bert_zh_L-12_H-768_A-12/1"
module_url="https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/1"
bert_layer = hub.KerasLayer(module_url, trainable=True)

#module_url="https://tfhub.dev/tensorflow/albert_en_xlarge/1"
#bert_layer = hub.KerasLayer(module_url, trainable=True)

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.01 µs


配置一个token工具，将根据BERT词表中的词把文本转化为一些列的token

In [0]:
#bert of tf-hub
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)


#en albert of tf-hub
#sp_model_file = bert_layer.resolved_object.sp_model_file.asset_path.numpy()
#tokenizer = tokenization.FullSentencePieceTokenizer(sp_model_file)

In [0]:
tokenizer.tokenize("学习自然语言处理的BERT模型")

['学', '习', '自', '然', '语', '言', '处', '理', '的', 'BE', '##RT', '模', '型']

## 2.3 训练模型

实现文本的编码

In [0]:
train_input = bert_encode(X_train.astype(str), tokenizer, max_len=200)
val_input = bert_encode(X_val.astype(str), tokenizer, max_len=200)
test_input = bert_encode(test['content'].astype(str), tokenizer, max_len=200)

39327     操他母亲了，打起仗来是不是还要看战斗机尾气符合不符合国5国6啊？我就不说其它国家的技术标准和...
17360                            我是真不明白那些瞒报行程瞒报发热的人脑子里是??吗?
84092     【武汉疫情背后的一条时间线】1.2019年12月30日，网上流传出一份盖有武汉卫健委公章的红...
4392      2020年新年快乐~今年是过的最人心惶惶的一个年，也希望新的病毒疫苗能尽快研发出来~希望一线...
110839    #抗疫一线群英谱##疫情当前警察不退#我是战士，我能坚持#战时政治建警# #抗击新型肺炎第一...
                                ...                        
40951     #请山西公布肺炎详细数据#请尽早公布运城有没有确诊和疑似的患者，听很多人说运城确诊了一例，还...
156791    #最新疫情地图#自己体温继续36.5正常，每天测试三次都正常。我因为得过胰腺炎，巴瑞特食管，...
156610    【国家卫健委：#全国疫情防控效果已经显现#】#滨海资讯#国家卫健委发言人米锋表示，截至2月1...
31071        大哥，叫你们下台不是因为你们封城啊，是因为你们渎职啊，懒政啊，不作为啊，你怎么这么拎不清呢？
3460      心情沉重也许这就是现实中的武汉吧，现在只能祈求一切顺利渡过，祈求病毒快点结束，祈求好友父母渡...
Name: content, Length: 142858, dtype: object
1312      李文亮不就是前车之鉴，呵呵。。 肯定不能进去！站着进去，跪着出来！ 派出所到底是什么地方，主...
65801     【官方回应较大数量发热病人未隔离收治：居家隔离的有20629人】湖北武汉仍有较大数量发热病人...
132330    #一封给2020的心愿单#国泰民安??????张艺兴我爱的人爱我的人身体健康事事顺心张艺兴四...
73                                        好烦躁。小宝发烧了2萍乡·芦溪县?
41319                         每天都在看古田有没有新增病例希望早点解封我想回家.

In [0]:
train_labels = train['target'].astype(int)+1

In [0]:
y_train = y_train.astype(int)+1
y_val= y_val.astype(int)+1

In [0]:
model = build_model(bert_layer, max_len=200)
met=Metrics([val_input,y_val])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 200)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 200)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 200)]        0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 768), (None, 177853441   input_word_ids[0][0]             
                                                                 input_mask[0][0]             

In [0]:
a=[train_input[0][:100],train_input[1][:100],train_input[2][:100]]
validation_data=[train_input[0][100:150],train_input[1][100:150],train_input[2][100:150]]
validation_target= train_labels[100:150]     
validation_data2 = [validation_data,validation_target] 

In [0]:
train_history = model.fit(
    train_input, y_train,
    epochs=5,
    batch_size=32,
    callbacks=[met]
)

Train on 142858 samples
Epoch 1/5
142858/142858 [==============================] - 3522s 25ms/sample - loss: 0.6930 - sparse_categorical_accuracy: 0.6964
Epoch 2/5
142858/142858 [==============================] - 3506s 25ms/sample - loss: 0.6051 - sparse_categorical_accuracy: 0.7406
Epoch 3/5
142858/142858 [==============================] - 3505s 25ms/sample - loss: 0.5562 - sparse_categorical_accuracy: 0.7630
Epoch 4/5
142858/142858 [==============================] - 3506s 25ms/sample - loss: 0.5136 - sparse_categorical_accuracy: 0.7819
Epoch 5/5
142858/142858 [==============================] - 3506s 25ms/sample - loss: 0.4731 - sparse_categorical_accuracy: 0.8006


In [0]:
model.save_weights(path+'bert_model_epoch5.h5')

In [0]:
test_pred = model.predict(test_input)

In [0]:
predictions = np.argmax(test_pred, axis=-1)-1
print(predictions)

[ 0 -1 -1 ...  0  0  1]


In [0]:
submit=pd.read_csv(path+'submit_example.csv')
submit['y']=predictions
submit.to_csv(path+'submission.csv', index=False)

## 2.4 加载模型之后继续训练

In [0]:
model_continue = build_model(bert_layer, max_len=200)
model_continue.load_weights(path+'bert_model_epoch8.h5')
train_history = model_continue.fit(
    train_input, y_train,
    epochs=3,
    batch_size=32,
    callbacks=[met]
)

Train on 142858 samples
Epoch 1/3
142858/142858 [==============================] - 3513s 25ms/sample - loss: 0.4380 - sparse_categorical_accuracy: 0.8162
Epoch 2/3
142858/142858 [==============================] - 3506s 25ms/sample - loss: 0.4049 - sparse_categorical_accuracy: 0.8317
Epoch 3/3
142858/142858 [==============================] - 3506s 25ms/sample - loss: 0.3762 - sparse_categorical_accuracy: 0.8447


In [0]:
model_continue.save_weights(path+'bert_model_epoch8.h5')
test_pred = model_continue.predict(test_input)
predictions = np.argmax(test_pred, axis=-1)-1
print(predictions)
submit=pd.read_csv(path+'submit_example.csv')
submit['y']=predictions
submit.to_csv(path+'submission.csv', index=False)

[ 0 -1  0 ...  0  0  0]


改进思路：  
删除文本中的人名（完成）  
将BERT模型换成ALBERT模型